In [1]:



import logging
import numpy as np

import matplotlib.pyplot as plt
from importlib import reload
import random

import tensorflow as tf
import importlib
import MCAgent as agent
import Model as Con4
import All_In_One_environment as env
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

2021-12-08 15:04:42.740860: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#local imports reload
importlib.reload(agent)
importlib.reload(Con4)
importlib.reload(env)

<module 'All_In_One_environment' from '/home/jevans40/AI/AIClass2021/Project/Group2-s_UW_Intro_AI_Project/Project_attempt_one/Project_attempt_one/All_In_One_environment.py'>

In [3]:
#settings:
#Settings
inputDim = (6,7,2)
outputDim = (7)
filterSize = 20
kernelDim = (4,4)
convLayers = 4
regConst = 0.0001
learningRate = 0.001
iteration = 1

Memory = []

In [4]:
def RunTwoAgents(Agent1,Agent2, Games, Verbosity = 0):
    memories = []
    for game in range(Games):
        print("Playing game {} out of {}".format(game,Games))
        winner = PlayGame(Agent1,Agent2,Verbosity)
        memories += Agent1.GetMemory()
        memories += Agent2.GetMemory()
        Agent1.ClearMemory()
        Agent2.ClearMemory()   

        tmp = Agent2
        Agent2 = Agent1
        Agent1 = tmp
    return memories
    #Return memory to train on

#Verbosity controls how much information to output
#0: None
#1: Winner/Loser
#2: Memory Output
def PlayGame(Agent1,Agent2,Verbosity=0):
    Agent1.SetPlayer(-1)
    Agent2.SetPlayer(1)
    done = False
    memories = []
    newGame = env.ConnectFour(1)
    num = random.randint(0,6)
    newGame.step(num,1)
    while not done:
        action = Agent1.GetAction(newGame)
        #Add some level of randomness so that we can get more varied cases
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent1.player)
        memories.append(action)
        if Verbosity > 1:
            print("Player -1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player -1 has won the game!")
            return -1

        action = Agent2.GetAction(newGame)
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent2.player)
        if Verbosity > 1:
            print("Player 1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player 1 has won the game!")
            return 1
        if Verbosity > 2:
            print(np.array(newGame.get_current_state()).reshape(6,7))
    
    #Return game result

def TrainModel(Memories, toFitAgent):
    for i in range(5):
        minbatch = random.sample(Memories,min(250,len(Memories)))

        trainingStates = np.array([Con4.ReshapeToModel(item[0],item[3]) for item in Memories])
        trainingTargets = np.array([item[1] for item in Memories])

        fit = toFitAgent.model.fit(trainingStates,trainingTargets,epochs=5,verbose=1,validation_split=0,batch_size = 32)

        toFitAgent.model.save("ModelCheckPointStart")
    return round(fit.history["loss"][0],4)

In [5]:
testModel = Con4.CN4Model(inputDim,outputDim,filterSize,kernelDim,regConst,convLayers,learningRate)
testModel.InitModel()


2021-12-08 15:04:52.350452: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-08 15:04:52.606391: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1f:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-08 15:04:52.606650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:1f:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.845GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.23GiB/s
2021-12-08 15:04:52.606706: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-08 15:04:52.608034: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-08 15:04:52.609507: I tensorflow/stream_executo

In [6]:
Agent1 = agent.ReinforcementAgent(testModel,-1)
Agent2 = agent.ReinforcementAgent(testModel,1)

In [ ]:
Mem = RunTwoAgents(Agent1, Agent2,5, Verbosity=2)
print(Mem)

In [ ]:
TrainModel(Mem,testModel)

In [11]:
while True:
    Mem = RunTwoAgents(Agent1, Agent2,10, Verbosity=0)
    TrainModel(Mem,testModel)

Playing game 0 out of 10
delta
[ 0.01072533 -0.02675193 -0.01225484 -0.0264328  -0.02751965  0.05535398
  0.02705562]
delta
[ 0.02549717  0.02533317  0.02579156 -0.03340615  0.02340932 -0.03342273
 -0.03296753]
delta
[ 0.02351898  0.01751158 -0.01026943 -0.00918463 -0.00306088 -0.00880216
 -0.00955532]
delta
[ 0.01835372  0.00259467 -0.00741612  0.00410751  0.00144235 -0.01662845
 -0.00228767]
delta
[ 0.05509617  0.07912091  0.01451905 -0.0487223   0.06376426 -0.10348373
 -0.06003796]
delta
[ 0.00323847 -0.00729183  0.01161502  0.01328415 -0.01566148 -0.00650581
  0.00147674]
delta
[ 0.09456165  0.02943548  0.02876245 -0.04622017 -0.06335282  0.03087077
 -0.07383103]
delta
[ 0.00122847  0.00235381  0.00194089 -0.00385001  0.0102802  -0.00403886
 -0.00776122]
delta
[ 0.00474129  0.02297355  0.03114976 -0.03491671  0.01947091 -0.00884883
 -0.03438895]
delta
[-0.00055789 -0.0031701   0.00083174  0.00213976 -0.00394243  0.00202253
  0.00281925]
delta
[-0.00639614 -0.02317086  0.02227023  0

2021-12-08 15:15:22.988714: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9702
Epoch 2/5
6/6 [==============================] - 0s 7ms/step - loss: 1.9605
Epoch 3/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9623
Epoch 4/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9560
Epoch 5/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9538
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s 8ms/step - loss: 1.9453
Epoch 2/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9457
Epoch 3/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9437
Epoch 4/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9441
Epoch 5/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9429
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
6/6 [==============================] - 0s